<a href="https://colab.research.google.com/github/LeticiaGDornelas/FIAP-fase2-cap6/blob/main/FIAP_Fase2_Cap6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LeiteControl**

O projeto LeiteControl tem como objetivo desenvolver um sistema simples de gestão da produção diária de leite em uma fazenda de gado leiteiro.

O problema identificado é a falta de controle informatizado da produção individual das vacas, que atualmente é feita de forma manual, dificultando a análise do desempenho de cada animal, a detecção de quedas de produção e a tomada de decisões baseadas em dados. Essa ausência de rastreabilidade compromete a eficiência produtiva e pode gerar perdas financeiras.

Como solução, o sistema LeiteControl foi criado em Python, utilizando os conceitos de subalgoritmos, estruturas de dados, manipulação de arquivos e conexão com banco de dados Oracle.
O sistema permite cadastrar vacas, registrar a produção de leite diária, calcular médias, detectar quedas de produtividade e gerar relatórios automáticos em formato .txt e .json. Os dados também são armazenados no Oracle Database, garantindo persistência e integridade das informações.

Com essa abordagem, o LeiteControl promove uma gestão mais organizada, segura e eficiente da produção leiteira, alinhando tecnologia à realidade do agronegócio.

**1) Subalgoritmos (funções e procedimentos com parâmetros)**

In [1]:
# ======================================
# 1) SUBALGORITMOS — LEITECONTROL (VERSÃO INTERATIVA)
# ======================================

from __future__ import annotations
from typing import List, Dict, Tuple
import datetime as dt

# ---------- ESTRUTURAS BÁSICAS ----------
rebanho: List[Dict] = []

# ---------- PROCEDIMENTOS ----------
def cadastrar_vaca(rebanho: List[Dict], id_: int, brinco: str, raca: str, nascimento_iso: str) -> None:
    """Cadastra uma nova vaca no rebanho."""
    vaca = {
        "id": id_,
        "brinco": brinco,
        "raca": raca,
        "nascimento": nascimento_iso,
        "ordenhas": []  # lista de tuplas (data, litros)
    }
    rebanho.append(vaca)

def registrar_ordenha(rebanho: List[Dict], id_: int, data_iso: str, litros: float) -> None:
    """Registra a produção de leite (litros) em uma data específica."""
    v = obter_vaca(rebanho, id_)
    if v is not None:
        v["ordenhas"].append((data_iso, float(litros)))
    else:
        print(f"❌ Vaca com ID {id_} não encontrada.")

# ---------- FUNÇÕES ----------
def obter_vaca(rebanho: List[Dict], id_: int) -> Dict | None:
    for v in rebanho:
        if v["id"] == id_:
            return v
    return None

def media_leite_vaca(vaca: Dict) -> float:
    """Calcula a média diária de produção (litros/dia) de uma vaca."""
    if not vaca["ordenhas"]:
        return 0.0
    total = sum(l for _, l in vaca["ordenhas"])
    dias = len(set(d for d, _ in vaca["ordenhas"]))
    return total / (dias or 1)

def detectar_quedas(vaca: Dict, queda_pct: float = 0.20) -> List[Tuple[str, float]]:
    """Detecta quedas de produção superiores ao limite indicado (20% padrão)."""
    ords = sorted(vaca["ordenhas"], key=lambda x: x[0])
    alertas = []
    for i in range(1, len(ords)):
        data_ant, litros_ant = ords[i-1]
        data_atual, litros_atual = ords[i]
        if litros_ant > 0:
            queda = (litros_ant - litros_atual) / litros_ant
            if queda >= queda_pct:
                alertas.append((data_atual, round(queda * 100, 1)))  # em %
    return alertas

# ---------- MENU INTERATIVO ----------
def menu_principal():
    while True:
        print("\n🐄=== SISTEMA LEITECONTROL ===")
        print("1 - Cadastrar nova vaca")
        print("2 - Registrar produção de leite")
        print("3 - Consultar média de leite por vaca")
        print("4 - Detectar quedas de produção")
        print("5 - Listar vacas")
        print("0 - Sair")

        opcao = input("Escolha uma opção: ").strip()
        if opcao == "1":
            id_ = int(input("ID da vaca: "))
            brinco = input("Brinco: ")
            raca = input("Raça: ")
            nasc = input("Data de nascimento (AAAA-MM-DD): ")
            cadastrar_vaca(rebanho, id_, brinco, raca, nasc)
            print("✅ Vaca cadastrada com sucesso!")
        elif opcao == "2":
            id_ = int(input("ID da vaca: "))
            data = input("Data da ordenha (AAAA-MM-DD): ")
            litros = float(input("Litros produzidos: "))
            registrar_ordenha(rebanho, id_, data, litros)
            print("✅ Produção registrada!")
        elif opcao == "3":
            id_ = int(input("ID da vaca: "))
            v = obter_vaca(rebanho, id_)
            if v:
                media = media_leite_vaca(v)
                print(f"📊 Média de produção: {media:.2f} L/dia")
            else:
                print("❌ Vaca não encontrada.")
        elif opcao == "4":
            id_ = int(input("ID da vaca: "))
            v = obter_vaca(rebanho, id_)
            if v:
                alertas = detectar_quedas(v)
                if alertas:
                    print("⚠ Quedas detectadas:")
                    for data, q in alertas:
                        print(f"  - {data}: {q:.1f}%")
                else:
                    print("✅ Nenhuma queda significativa.")
            else:
                print("❌ Vaca não encontrada.")
        elif opcao == "5":
            if rebanho:
                for v in rebanho:
                    print(f"- ID {v['id']} | {v['brinco']} ({v['raca']})")
            else:
                print("📋 Nenhuma vaca cadastrada.")
        elif opcao == "0":
            print("Encerrando o sistema... 🐮")
            break
        else:
            print("Opção inválida!")

# ---------- EXECUÇÃO ----------
menu_principal()



🐄=== SISTEMA LEITECONTROL ===
1 - Cadastrar nova vaca
2 - Registrar produção de leite
3 - Consultar média de leite por vaca
4 - Detectar quedas de produção
5 - Listar vacas
0 - Sair
Escolha uma opção: 1
ID da vaca: 002
Brinco: 125
Raça: sol
Data de nascimento (AAAA-MM-DD): 2025-09-06
✅ Vaca cadastrada com sucesso!

🐄=== SISTEMA LEITECONTROL ===
1 - Cadastrar nova vaca
2 - Registrar produção de leite
3 - Consultar média de leite por vaca
4 - Detectar quedas de produção
5 - Listar vacas
0 - Sair
Escolha uma opção: 1
ID da vaca: 003
Brinco: 127
Raça: sol
Data de nascimento (AAAA-MM-DD): 2010-05-08
✅ Vaca cadastrada com sucesso!

🐄=== SISTEMA LEITECONTROL ===
1 - Cadastrar nova vaca
2 - Registrar produção de leite
3 - Consultar média de leite por vaca
4 - Detectar quedas de produção
5 - Listar vacas
0 - Sair
Escolha uma opção: 0
Encerrando o sistema... 🐮


**2) Estruturas de dados (lista, tupla, dicionário, tabela de memória)**

In [2]:
# =======================================================
# 2) ESTRUTURAS DE DADOS — MODELO ALINHADO AO LEITECONTROL
# =======================================================

from typing import List, Dict, Tuple

# Tabela de memória principal (se já existir da Parte 1, reutiliza):
try:
    rebanho  # type: ignore
except NameError:
    rebanho: List[Dict] = []

def nova_vaca(id_: int, brinco: str, raca: str, nasc_iso: str) -> Dict:
    """Cria o dicionário da vaca no formato usado pelo sistema."""
    return {
        "id": id_,
        "brinco": brinco,
        "raca": raca,
        "nascimento": nasc_iso,   # "YYYY-MM-DD"
        "ordenhas": []            # List[Tuple[data_iso, litros]]
    }

# EXEMPLO (pode comentar se já cadastrou via menu da Parte 1)
if not any(v.get("id") == 101 for v in rebanho):
    rebanho.append(nova_vaca(101, "A101", "Holandês", "2022-08-15"))
    rebanho[-1]["ordenhas"].extend([
        ("2025-10-10", 20.5),
        ("2025-10-11", 19.8),
        ("2025-10-12", 14.0),
    ])

print(f"Total de vacas no rebanho: {len(rebanho)}")
print("Primeira vaca (exemplo):", rebanho[0] if rebanho else "—")


Total de vacas no rebanho: 3
Primeira vaca (exemplo): {'id': 2, 'brinco': '125', 'raca': 'sol', 'nascimento': '2025-09-06', 'ordenhas': []}


**3) Manipulação de arquivos (texto e JSON)**

In [3]:
# ===========================================
# 3) ARQUIVOS — SALVAR/CARREGAR JSON + RELATÓRIO TXT
# ===========================================

import json
from typing import List, Dict, Tuple
from datetime import date

DEFAULT_JSON = "/content/rebanho.json"
DEFAULT_TXT  = "/content/relatorio_leitecontrol.txt"

def salvar_json(rebanho: List[Dict], caminho: str = DEFAULT_JSON) -> None:
    """Salva o rebanho (lista de dicionários) em JSON."""
    with open(caminho, "w", encoding="utf-8") as f:
        json.dump(rebanho, f, ensure_ascii=False, indent=2)
    print(f"✅ JSON salvo em {caminho}")

def carregar_json(caminho: str = DEFAULT_JSON) -> List[Dict]:
    """Carrega o rebanho a partir de um JSON."""
    with open(caminho, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"📥 JSON carregado de {caminho}")
    return data

def _media_vaca_local(v: Dict) -> float:
    """Média (L/dia) calculada sem depender da Parte 1."""
    if not v.get("ordenhas"):
        return 0.0
    litros = sum(l for _, l in v["ordenhas"])
    dias = len(set(d for d, _ in v["ordenhas"]))
    return litros / (dias or 1)

def _quedas_local(v: Dict, queda_pct: float = 0.20) -> List[Tuple[str, float]]:
    """Quedas ≥ queda_pct; retorna lista (data, pct%)."""
    ords = sorted(v.get("ordenhas", []), key=lambda x: x[0])
    out = []
    for i in range(1, len(ords)):
        d_ant, l_ant = ords[i-1]
        d_atual, l_atu = ords[i]
        if l_ant > 0:
            q = (l_ant - l_atu) / l_ant
            if q >= queda_pct:
                out.append((d_atual, round(q*100, 1)))
    return out

def gerar_relatorio_txt(rebanho: List[Dict], caminho: str = DEFAULT_TXT,
                        queda_pct: float = 0.20) -> None:
    """Relatório com média por vaca e alertas de queda."""
    linhas = [f"RELATÓRIO — LEITECONTROL ({date.today().isoformat()})", ""]
    total_litros, total_dias = 0.0, 0

    for v in rebanho:
        media_v = _media_vaca_local(v)
        linhas.append(f"- {v['brinco']} ({v['raca']}) — média: {media_v:.2f} L/dia")

        # agrega para média do rebanho
        litros_hist = sum(l for _, l in v.get("ordenhas", []))
        dias_hist   = len(set(d for d, _ in v.get("ordenhas", [])))
        total_litros += litros_hist
        total_dias   += dias_hist

        quedas = _quedas_local(v, queda_pct=queda_pct)
        if quedas:
            for d, q in quedas:
                linhas.append(f"   ⚠ Queda em {d}: {q:.1f}%")
        else:
            linhas.append("   ✓ Sem quedas relevantes")

    media_rebanho = (total_litros / total_dias) if total_dias else 0.0
    linhas += ["", f"Média do rebanho: {media_rebanho:.2f} L/dia"]

    with open(caminho, "w", encoding="utf-8") as f:
        f.write("\n".join(linhas))
    print(f"📝 Relatório salvo em {caminho}")

# DEMO rápido (com a variável 'rebanho' da Parte 2/1)
if 'rebanho' in globals() and isinstance(rebanho, list):
    salvar_json(rebanho)
    rebanho_lido = carregar_json()
    gerar_relatorio_txt(rebanho_lido)
else:
    print("Defina 'rebanho' antes (rodar Partes 1 e 2).")


✅ JSON salvo em /content/rebanho.json
📥 JSON carregado de /content/rebanho.json
📝 Relatório salvo em /content/relatorio_leitecontrol.txt


**4) Conexão com banco de dados (Oracle, Colab — modo thin)**

In [4]:
# ==========================================================
# 4) ORACLE — CONEXÃO, SCHEMA, SYNC COM 'rebanho' E CONSULTAS
# ==========================================================

!pip -q install oracledb

import oracledb
from getpass import getpass

# 1) Conexão ------------------------------------------------
HOST = input("HOST (ex: oracle.fiap.com.br): ").strip() or "oracle.fiap.com.br"
PORT = input("PORT (ex: 1521): ").strip() or "1521"
SERVICE = input("SERVICE_NAME (ex: ORCL): ").strip() or "ORCL"
USER = input("Usuário Oracle: ").strip()
PASSWORD = getpass("Senha Oracle: ")
DSN = f"{HOST}:{PORT}/{SERVICE}"

def conectar_oracle(user: str, password: str, dsn: str):
    try:
        conn = oracledb.connect(user=user, password=password, dsn=dsn)
        print("✅ Conexão Oracle estabelecida.")
        return conn
    except Exception as e:
        print("❌ Erro ao conectar:", e)
        return None

conn = conectar_oracle(USER, PASSWORD, DSN)

# 2) Se não houver 'rebanho' em memória, tenta carregar do JSON
if 'rebanho' not in globals():
    try:
        import json
        with open("/content/rebanho.json", "r", encoding="utf-8") as f:
            rebanho = json.load(f)
        print("📥 'rebanho' carregado do JSON para sincronização.")
    except Exception as _:
        rebanho = []
        print("⚠ Não há 'rebanho' em memória nem JSON disponível.")

# 3) DDL + Sync --------------------------------------------
if conn:
    cur = conn.cursor()

    # Cria tabelas (idempotente)
    cur.execute("""
    BEGIN
      EXECUTE IMMEDIATE 'CREATE TABLE VACAS (
        ID NUMBER PRIMARY KEY,
        BRINCO VARCHAR2(20),
        RACA VARCHAR2(40),
        NASCIMENTO DATE
      )';
    EXCEPTION WHEN OTHERS THEN IF SQLCODE != -955 THEN RAISE; END IF;
    END;""")

    cur.execute("""
    BEGIN
      EXECUTE IMMEDIATE 'CREATE TABLE ORDENHAS (
        ID_VACA NUMBER,
        DATA_ORDENHA DATE,
        LITROS NUMBER(10,2),
        CONSTRAINT FK_ORD_VACA FOREIGN KEY (ID_VACA) REFERENCES VACAS(ID)
      )';
    EXCEPTION WHEN OTHERS THEN IF SQLCODE != -955 THEN RAISE; END IF;
    END;""")

    # Upsert de VACAS + replace das ORDENHAS (simples p/ demo)
    for v in rebanho:
        cur.execute("""
            MERGE INTO VACAS t
            USING (SELECT :id AS id FROM dual) s
            ON (t.ID = s.id)
            WHEN NOT MATCHED THEN
              INSERT (ID, BRINCO, RACA, NASCIMENTO)
              VALUES (:id, :brinco, :raca, TO_DATE(:nasc,'YYYY-MM-DD'))
        """, {"id": v["id"], "brinco": v["brinco"], "raca": v["raca"], "nasc": v["nascimento"]})

        cur.execute("DELETE FROM ORDENHAS WHERE ID_VACA = :id", {"id": v["id"]})
        for data_iso, litros in v.get("ordenhas", []):
            cur.execute("""
                INSERT INTO ORDENHAS (ID_VACA, DATA_ORDENHA, LITROS)
                VALUES (:id, TO_DATE(:dt,'YYYY-MM-DD'), :l)
            """, {"id": v["id"], "dt": data_iso, "l": litros})

    conn.commit()
    print("📤 Dados sincronizados com Oracle.")

    # 4) Consultas úteis ------------------------------------
    print("\n📊 Médias diárias por vaca (Oracle):")
    cur.execute("""
        SELECT v.BRINCO,
               ROUND(AVG(sub.LITROS_DIA), 2) AS MEDIA_L_DIA
        FROM VACAS v
        JOIN (
            SELECT ID_VACA, DATA_ORDENHA, SUM(LITROS) AS LITROS_DIA
            FROM ORDENHAS
            GROUP BY ID_VACA, DATA_ORDENHA
        ) sub ON sub.ID_VACA = v.ID
        GROUP BY v.BRINCO
        ORDER BY v.BRINCO
    """)
    for row in cur.fetchall():
        print(row)

    print("\n🔻 Abaixo da média do rebanho (Oracle):")
    cur.execute("""
        WITH por_vaca AS (
            SELECT v.ID, v.BRINCO, AVG(sub.LITROS_DIA) AS MEDIA_VACA
            FROM VACAS v
            JOIN (
                SELECT ID_VACA, DATA_ORDENHA, SUM(LITROS) AS LITROS_DIA
                FROM ORDENHAS
                GROUP BY ID_VACA, DATA_ORDENHA
            ) sub ON sub.ID_VACA = v.ID
            GROUP BY v.ID, v.BRINCO
        ),
        media_rebanho AS (
            SELECT AVG(MEDIA_VACA) AS MEDIA_REB FROM por_vaca
        )
        SELECT p.BRINCO, ROUND(p.MEDIA_VACA,2) AS MEDIA_VACA,
               ROUND(m.MEDIA_REB,2) AS MEDIA_REBANHO
        FROM por_vaca p CROSS JOIN media_rebanho m
        WHERE p.MEDIA_VACA < m.MEDIA_REB
        ORDER BY p.MEDIA_VACA ASC
    """)
    for row in cur.fetchall():
        print(row)

    cur.close()
    conn.close()
    print("\n🔌 Conexão encerrada.")
else:
    print("Sem conexão. Execute esta célula após informar as credenciais corretas.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 35.4 MB/s eta 0:00:00
HOST (ex: oracle.fiap.com.br): oracle.fiap.com.br
PORT (ex: 1521): 1521
SERVICE_NAME (ex: ORCL): ORCL
Usuário Oracle: RM568172
Senha Oracle: ··········
✅ Conexão Oracle estabelecida.
📤 Dados sincronizados com Oracle.

📊 Médias diárias por vaca (Oracle):
('A101', 18.1)
('A123', 18.1)

🔻 Abaixo da média do rebanho (Oracle):

🔌 Conexão encerrada.
